(section: tangent_hypothesis)=
# Characterizing the tangent hypothesis

We now have a diagnostic of the maximum intensity arc of the image as well as the maximum gradient. We can compare these arcs with the projection of the tangent direction of the magnetopause surface we have extracted from the simulation, to understand if we can derive a correlation. 

(numerical_surface_projection)=
## Projecting the surface 

Since we have extracted the numerical surface in spherical coordinates, we can also find the points that correspond to the tangent direction and project them into the \ac{FOV} of the instrument. Let's suppose that for a particular image, the satellite is located at the position $\vec{r}_{sat}=(x_{sat}, y_{sat}, z_{sat})$ in the \ac{GSE} system. The tangent points of the surface for this location will satisfy the condition:

$$
    \vec{n} \cdot (\vec{r}_{point}-\vec{r}_{sat} )= 0
$$(eq:tangent_eq)

where $\vec{r}_{point}$ is the position of each point of the numerical surface and $\vec{n}$ is the normal vector of the surface for each point, in the GSE system. We can get $\vec{r}_{point}$ by simply applying the transformation from the spherical system to the cartesian as defined in {ref}`coordinate`. 
To get the normal vectors, we calculate the gradient of the surface with respect to $\theta$ and $\phi$ and normalize the corresponding vectors. We can calculate the normals for each point through their cross product:

$$
    \vec{n} = \frac{\partial \vec{r}}{\partial \theta} \times \frac{\partial \vec{r}}{\partial \phi}
$$

To get the tangent curve, we simply solve Equation {eq}`eq:tangent_eq` numerically, to get the points that satisfy it. In our case, we applied a simple mask with an arbitrary threshold of $0.05^\circ$ tolerance, as a first approximate and fast solution.

\begin{figure}[bth]
    \myfloatalign

    \subfloat[Calculation of tangent points to satellite.]
    {
    \label{fig:schematic}
    \includegraphics[width=0.45\textwidth]{gfx/projection/tangent_points.png}} 
    \quad
    \subfloat[Tangent points in GSE.]
    {
    \label{fig:tangent_points}
    \includegraphics[width=0.45\textwidth]{gfx/projection/3D.png}} 
    \quad

    \caption{Extraction of tangent point to satellite for a numerical surface.}  
    \label{fig:tangent}
\end{figure}

The orientation of the satellite is described by three vectors, giving each base vector of the satellite coordinate system, expressed in the \ac{GSE} frame. To move from one system to another, it is sufficient to define the rotation matrix through these vectors and apply a translation according to the position of the satellite, meaning the origin of the satellite's coordinate system. We construct the rotation matrix:

$$
    R = \begin{bmatrix} | & | & | \\ \mathbf{e}_x^{sat} & \mathbf{e}_y^{sat} & \mathbf{e}_z^{sat} \\ | & | & | \end{bmatrix}
$$

The relation between a vector in the GSE frame and the same vector in the satellite frame is:

$$
    \vec{r}_{GSE} = R \cdot \vec{r}_{SF} + \vec{t}
$$
where $r_{SF}$ is the vector of a point in the satellite frame and $\vec{t}$ is the translation vector. Therefore, the transformation of a point in \ac{GSE} to the satellite frame is performed by the following operation:

$$
    \vec{v}_{sat} = R^\top \cdot (\vec{v}_{GSE} - \vec{t})
$$
In order to project this in the \ac{SXI} coordinates we need to compute the angle of each point from the z axis (boresight) along the x and y direction of the satellite frame:

$$
    azimuth = \arctan{\frac{x}{z}}
$$

$$
    elevation = \arctan{\frac{y}{z}}
$$
And limit this to the \ac{FOV} of the imager, with $azimuth \in [-7.8,7.7]^\circ$ and $elevation\in[-13.2,13.2]^\circ$. We can also define a coordinate transformation from the \ac{SXI} degree coordinates to the indexes of the image, by mapping these angles to grid indices $i_{\text{az}}$ and $i_{\text{el}}$:

$$
    i_{\text{az}} = \text{round} \left( \frac{\theta_{\text{az}} - \text{Az}{\min}}{\text{Az}{\max} - \text{Az}{\min}} \cdot (N{\text{az}} - 1) \right)
$$
$$
    i_{\text{el}} = \text{round} \left( \frac{\theta_{\text{el}} - \text{El}{\min}}{\text{El}{\max} - \text{El}{\min}} \cdot (N{\text{el}} - 1) \right)
$$

where $N_{\text{az}}$ and $N_{\text{el}}$ are the number of azimuth and elevation grid points, respectively. To ensure the indices stay within valid bounds we also clip to the minimum of the grid:

$$i_{\text{az}} = \min\left(\max\left(i_{\text{az}}, 0\right), N_{\text{az}} - 1\right)$$
$$i_{\text{el}} = \min\left(\max\left(i_{\text{el}}, 0\right), N_{\text{el}} - 1\right)$$

% image of typical projection

(tangent_results)=
## Results: Characterizing the Tangent Hypothesis

We can now derive diagnostics about the correlation of the maximum intensity curve of the image and the projected tangent points of the surface. We do this by calculating the \ac{RMSE} between the two curves of each image. By taking a sequence of images over some key orbits we can understand how this error evolves depending on the geometry of observation. We run this test for a full orbit ($T = 50.3 \,hours$) per month, from June 2025 to May 2026. As described in \autoref{SMILE}, the orbit has an inclination of $73^\circ$, however the orientation of this inclination precesses and thus the geometry of observation changes seasonally. This in turn affects the projection of the magnetopause to the \ac{FOV} as well as the tracing of the maximum intensity arc. We will look into this seasonal dependency of the tangent hypothesis, and will point out its limitations.

\begin{figure}[bth]
    \myfloatalign
    \includegraphics[width=\textwidth]{gfx/tangent_hypothesis/June_RMSE.png}
    \quad
    \subfloat[Index 02]{\label{fig:jun02}
    \includegraphics[width=0.2\textwidth]{gfx/tangent_hypothesis/02June.png}}
    \quad
    \subfloat[Index 50]{\label{fig:jun50}
    \includegraphics[width=0.2\textwidth]{gfx/tangent_hypothesis/50June.png}}
    \quad
    \subfloat[Index 86]{\label{fig:jun86}
    \includegraphics[width=0.2\textwidth]{gfx/tangent_hypothesis/86June.png}}
    \quad
    \subfloat[Index 99]{\label{fig:jun99}
    \includegraphics[width=0.2\textwidth]{gfx/tangent_hypothesis/99June.png}}
    \quad
    \caption{Evolution of RMSE over time elapsed from 2 June 2025 00:00 UTC and key snapshots. Magnetopause projection is indicated in blue while the red curve indicates the detected maximum intensity arc.}

    \label{fig:JuneRMSE}
\end{figure}

In \autoref{fig:JuneRMSE} we see the evolution of the \ac{RMSE} over a summer orbit in June 2025. The agreement between the projection of the magnetopause surface, indicated with the blue points, and the maximum intensity arc of the image, indicated with the red line, is close to 0.1 RE for most of the orbit, going sufficiently below the 0.5 RE error that was set as a requirement for the \ac{SXI} instrument. An example of a good agreement is shown in the snapshot of \autoref{fig:jun50} and \autoref{fig:jun99}. We also observe cases where the error peaks, as in \autoref{fig:jun02}, where although the projection seems correct, the extraction of the maximum intensity line fails to capture the intended curve. Finally, we show an example of the satellite entering the magnetopause in \autoref{fig:jun86}, where no curve can be detected since the emissivity inside the magnetosphere is zero. In such cases we discard the measurement since this is a clear limitation of the \ac{TFA}.


\begin{figure}[bth]
    \myfloatalign
    \includegraphics[width=\textwidth]{gfx/tangent_hypothesis/Nov_RMSE.png}
    \quad
    \subfloat[Index 23]{
    \includegraphics[width=0.2\textwidth]{gfx/tangent_hypothesis/23Nov.png}}
    \quad
    \subfloat[Index 50]{\label{fig:50nov}
    \includegraphics[width=0.2\textwidth]{gfx/tangent_hypothesis/50Nov.png}}
    \quad
    \subfloat[Index 62]{\label{fig:62nov}
    \includegraphics[width=0.2\textwidth]{gfx/tangent_hypothesis/62Nov.png}}
    \quad
    \subfloat[Index 90]{\label{fig:90nov}
    \includegraphics[width=0.2\textwidth]{gfx/tangent_hypothesis/90Nov.png}}
    \quad
    \caption{Evolution of RMSE over time elapsed from 1st November 2025 00:00 UTC and key snapshots.}

    \label{fig:NovRMSE}
\end{figure}

Moving to the winter months, we no longer have problems with tracing the maximum intensity line, however certain \ac{POV}s show a clear problem with the magnetopause surface and its projection to the \ac{FOV}. \autoref{fig:50nov} and \autoref{fig:62nov} show two cases where the projection shows no correlation with the expected shape. The inner arc is a projection of the indentation of the cusps. Our grid is quite sparse for such a rapid identation, leading to not representative gradients of the surface, at the point of the discontinuity. However, this also shows our inability to correlate the projection of the cusps with a distinct geometric feature of the images. Although the cusps are highly luminous, they are also localized, meaning that their emission is not integrated over a significant portion of the \ac{LOS}. This results in structures that are noticeable in the integrated image, but not prominent or consistent enough to allow for the correlation of their projection with their position. In any case, the concept of the tangent hypothesis has its foundation on the integration of a relatively homogeneous emission over the maximum path - excluding thus bright but localized features.

The outer arc projection seen in \autoref{fig:50nov} is more difficult to interpret, and the orbit of the satellite during the winter months should be taken into account. To understand this projection, we shall take a look at the viewing angle of the 3D magnetopause surface, as shown in the summary of the projection problem in \autoref{fig:POV}. The interpretation of the inner arc clearly agrees with the projection of cusps here. The outer tangent to the surface however, seems to severely overestimate the position of the maximum intensity arc. This can be explained by going back to the conclusion \citeauthor*{andi_2024} reached in his paper, where he examined a spherical homogeneously emitting surface and found the tangent hypothesis to hold true. However, when he introduced gradients to the emission, making some regions more luminous than others, the maximum intensity arc consistently underestimated the position of the tangent arc \cite{andi_2024}. It is also known that the subsolar point will be significantly more luminous than higher latitudes, due to the flux of the solar wind. The contribution from the integration of the high latitude emission is not enough to counter the emission gradient near the subsolar point, leading to the underestimation of the tangent position when the satellite exceeds a certain latitude. This is reflected in the correlation of this phenomenon with the winter orbits (\autoref{fig:orbitNov}), as well as with the polar angle of the position of the satellite. We show this correlation in \autoref{theta-RMSE}, and we will further discuss it below.

\begin{figure}[bth]
    \myfloatalign
    \includegraphics[width=\textwidth]{gfx/tangent_hypothesis/Apr_RMSE.png}
    \quad
    \subfloat[Index 40]{
    \includegraphics[width=0.2\textwidth]{gfx/tangent_hypothesis/40Apr.png}}
    \quad
    \subfloat[Index 58]{
    \includegraphics[width=0.2\textwidth]{gfx/tangent_hypothesis/58Apr.png}}
    \quad
    \subfloat[Index 85]{ \label{fig:85aug}
    \includegraphics[width=0.2\textwidth]{gfx/tangent_hypothesis/85Apr.png}}
    \quad
    \subfloat[Index 95]{
    \includegraphics[width=0.2\textwidth]{gfx/tangent_hypothesis/95Apr.png}}
    \quad
    \caption{Evolution of RMSE over time elapsed from 1st April 2026 00:00 UTC and key snapshots.}

    \label{fig:AprRMSE}
\end{figure}

Looking into another summer orbit, during April 2026 (\autoref{fig:AprRMSE}), we observe similar behavior with June 2025, where the projection of the surface displays no surprising components, however the extraction of the maximum intensity arc oftentimes fails. This is strongly correlated with the inclination during these months and the resulting \ac{POV}s. \autoref{fig:POV} shows the corresponding viewing angles of the 3D surface for arc extraction problem. The contribution to the intensity of every part of the surface, makes it difficult to track a single and distinct maximum intensity arc. This is clearly reflected in the integrated image of \autoref{fig:85aug}, where the previously distinct arc is no longer a thin line. A representative orbit during the summer months is shown in \autoref{fig:orbitApr}, where X is the axis towards the sun in the \ac{GSE} system.

\begin{figure}
    \centering
    \includegraphics[width=0.85\textwidth]{gfx/tangent_hypothesis/projections.png}
    \caption{Viewing direction correlated with surface projection and image processing problems}
    \label{fig:POV}
\end{figure}

\begin{figure}[bth]
    \myfloatalign
    \subfloat[June 2025]{
    \includegraphics[width=0.3\textwidth]{gfx/tangent_hypothesis/June_orbit.png}}
    \quad
    \subfloat[November 2025]{\label{fig:orbitNov}
    \includegraphics[width=0.3\textwidth]{gfx/tangent_hypothesis/Nov_orbit.png}}
    \quad
    \subfloat[April 2026]{\label{fig:orbitApr}
    \includegraphics[width=0.3\textwidth]{gfx/tangent_hypothesis/Apr_orbit.png}}
    \quad
    \caption{Orbit precession and corresponding viewing angles for June, November and April.}
    \label{fig:orbits}
\end{figure}


While, correlating the above orbits, as well the orbits for each month that are not explicitly discussed here, we see a clear dependency on the polar angle of the satellite's position. In \autoref{theta-RMSE} we notice that the error rapidly increases when the satellite is in the nightside, while there is no detection below $50^\circ$ polar angle. The latter is to be expected, as we approach and enter the magnetopause for lower angles. We discussed previously the bias that the emission gradients could impose on the detection of the tangent direction. Here we see that for angles larger than $90^\circ$ - when the satellite is in the nightside - this bias becomes rapidly important and the tangent hypothesis fails. 


\begin{figure}[bth]
    \centering
    \myfloatalign
    \subfloat[RMSE dependency to the $\theta$ angle of the satellite position over a year of orbits.]
    {\label{theta-RMSE}
    \includegraphics[width=0.6\textwidth]{gfx/tangent_hypothesis/theta-RMSE.png}}
    \quad
    \subfloat[High satellite $\theta$ angle viewing geometry.]{
    \raisebox{0.7cm}{
    \label{Oct3Dview}
    \includegraphics[width=0.25\textwidth]{gfx/tangent_hypothesis/3DView.png}}}
    \caption{Orbital dependency of RMSE over a year.}
    \label{fig:theta_sat-RMSE}
\end{figure}

Looking into \autoref{fig:theta_sat-RMSE}, we see that this bias is not only attributed to the physical gradients, but also to the simulation restrictions. The simulation cube is restricted to $z\in[-10,10]\, RE$, meaning that any emission outside this range is not captured in the integrated images. At the same time the magnetopause surface is constructed directly from the emission data of the simulation cube, meaning that the projected surface is limited in $\theta\in[-65,65]^\circ$. For a full characterization of the high latitude angles, a simulation box to encapsulate all satellite positions and orbits will be necessary.

% \subsection{Discussion: Definition of the magnetopause}

% \label{definition_probelms}

We pointed out earlier that we cannot extract meaningful information from the projection of the cusps in the image using the \ac{TFA}. In other words, the use of a magnetopause model with strong cusp indentation does not represent the curve of the maximum intensity, that we want to fit. This opens the conversation on the definition of the magnetopause surface and the disagreement with the extracted surface, that we noticed in \autoref{fitting_models}. When trying to fit empirical models, it was evident that their shape could not describe the discontinuous surface of the maximum intensity, nor the discontinuity predicted by the pressure equilibrium equation \autoref{pressure_eq}. 

\citeauthor{nguyen4} re-enforces this disagreement by concluding that the Lin model overestimates the identation of the cusps \cite{nguyen4}, when for the case of our extracted surface, every model severely underestimates it. In the last paper of his series, he explains the selection criteria for the crossings in the cusp region. He indicates the existence of two barriers - the inner and the outer cusp - and chooses the latter to perform the statistical analysis. This may be correlated with the second maximum emissivity of the 3D cube in that region, and further tests on the surface extraction shall be performed to understand how the empirical models correlate with the simulation results. 
Further metrics shall also be tested, emphasizing the dependency on the electromagnetic fields instead of the emission. The extraction of the magnetopause through the current sheet would be of particular interest, since its theoretical shape seems to better align with the empirical models. Future efforts shall concentrate on extracting the outer boundary, rejecting the inner cusp and defining coherently the magnetopause surface.  